# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import numpy as np
import logging
import sys

# Logging
logging.basicConfig(
    level=logging.ERROR,
    format='%(asctime)s %(levelname)s \t %(message)s ',
    datefmt='%Y-%m-%d %H:%M:%S',
    stream=sys.stdout,
)
log = logging.getLogger('log')

# Improve view
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [2]:
# Function definitions

def quantify_data(df, type_choice=None, examples=10):
    """ DOC STRING"""
    # Set number of examples to be printed per value
    # examples = 10
    
    # If type_choice is set, only the dtypes provided will be analysed
    if type_choice is None:
        # If not set, we simple analyze numeric and string data and
        # print the result
        type_choice = ['all']
        print('Running Data Quantifier with parameter: ', ', '.join(type_choice),\
             ' and example threshhold is ', examples)
    else:
        print('Running Data Quantifier with parameter: ',  ', '.join(type_choice),\
             ' and example threshhold is ', examples)
    
    # Analysis section
    if (('all' in type_choice) or ('numbers' in type_choice)):
        # NUMERIC DATA ANALYSIS
        sub_df = df.select_dtypes(exclude=['object'])
        print('\nQuantifying NUMERIC data types in columns:\n',  ', '.join(sub_df.columns), '\n')
        # Get descriptive statistics
        stat_df = sub_df.describe()
        # Count missing values per column
        miss_df = pd.DataFrame.from_dict({'Missing': sub_df.isna().sum()})
        #miss_df = miss_df['Missing'].astype(int)
        #mis_val_cols = miss_df.loc[miss_df['Missing'] > 0].columns
        mis_val_cols = miss_df[miss_df > 0].dropna().index
        # Count unique values per column
        uniq_df = pd.DataFrame.from_dict({'Unique': sub_df.nunique()})
        #uniq_df = uniq_df['Unique'].astype(int)
        # Get list of example values for columns which have less than x unique values
        uni_val_cols = uniq_df[uniq_df <= examples].dropna().index
        uniq_df = uniq_df.transpose()
        miss_df = miss_df.transpose()
        stat_df = pd.concat([stat_df, uniq_df, miss_df])
        display(stat_df)
        print('Columns with missing values: ', ','.join(mis_val_cols), '\n')
        for unique_value_column in uni_val_cols:
            unique_values = df[unique_value_column].drop_duplicates()
            msg = 'Unique values in column \'{}\': \n'.format(unique_value_column)
            print(msg, unique_values.values, '\n')
        #print('Columns with missing values: ', ','.join(mis_val_cols))

    if (('all' in type_choice) or ('object' in type_choice)):
        # STRING DATA ANALYSIS
        sub_df = df.select_dtypes(exclude=['float64'])
        print('\nQuantifying NON-NUMERIC data types in columns:\n',  ', '.join(sub_df.columns))
        stat_df = pd.DataFrame.from_dict(data=dict(sub_df.dtypes), orient='index', columns=['Datatype'])
        stat_df['Lines'] = len(df)
        stat_df['Non-Null'] = df.count()
        stat_df['NaN'] = df.isna().sum()
        stat_df['Fill-%'] = df.count() / len(df) *100
        stat_df['Unique'] = df.nunique()
        stat_df['Uniq-%'] = stat_df['Unique'] / stat_df['Lines'] *100
        mis_val_cols = list(stat_df.loc[stat_df['Fill-%'] < 100].index)
        uni_val_cols = list(stat_df.loc[stat_df['Unique'] <= examples].index)
        display(stat_df.transpose())
        print('Columns with missing values: ', ','.join(mis_val_cols), '\n')
        for unique_value_column in uni_val_cols:
            unique_values = df[unique_value_column].drop_duplicates()
            msg = 'Unique values in column \'{}\': \n'.format(unique_value_column)
            print(msg, unique_values.values, '\n')
    print("\n\nData Quantification Done\n\n")

# Step 1 - Scoping and Data Gathering
**Task: Scope the Project and Gather Data**

*Identify and gather the data you'll be using for your project (at least two sources and more than 1 million rows). See Project Resources for ideas of what data you can use.*

*Explain what end use cases you'd like to prepare the data for (e.g., analytics table, app back-end, source-of-truth database, etc.)*


## Step 1a - General Scope and Data Gathering Description
The Udacity provided datasets for the Capstone Project include:
* I94 Immigration data from 2016 provided by U.S. Customs and Border Protection agency
* World Temperature Data
* U.S. cities demographic data
* An airport code table

Each dataset has been collected at least once for assessment. The findings are included in the following chapters of this notebook, even if the dataset is not used in Step 2.

Regarding the scope itself the following findings are relevant:
* **I94 Immigration data** is considered **in scope** regarding the following analytical tasks:
    * Develop a scalable automated extraction procedure using Spark Data Lake
    * Load and Transform the data into fact and dimension tables
    * Develop Airflow routines to manage the process
* **Airport Codes** are considered **in scope** and will be used
    * to enrich the immigration dataset with complete and updated values
* **World Temperature data** is considered **out of scope** since no analytics questions for this dataset in conjunction with immigration data could be identified _and_ the datasets' time periods do not overlap
* **Demographic data** is considered

**Approach to describe and gather data**

Descriptions for each dataset will be given in the sections below. Each description shall include:
1. A first read of the dataset using Python and Pandas default methods
1. "First Impression" notes about the extracted data
1. Analysis of dataset documentation, enclosed data dictionaries, etc.
1. Findings about Data Meaning, Quality, possible relationsships and definitions for
    1. Numeric columns (including missing values, uniqueness and descriptive statistics)
    1. Non-numeric columns

## Step 1b - I94 Dataset of U.S. Customs and Border Protection department

### A - I94 Immigration Dataset Description
The dataset provided contains immigration data provided by US immigration authorities. Data is collected via form **I94** and contains data about people travelling from and to the US on people who are either **non United States citizens** or **lawful permanent residents** in the US.

    “Form I-94, the Arrival-Departure Record Card, is a form used by the U.S. Customs and Border Protection (CBP) intended to keep track of the arrival and departure to/from the United States of people who are not United States citizens or lawful permanent residents (with the exception of those who are entering using the Visa Waiver Program or Compact of Free Association, using Border Crossing Cards, re-entering via automatic visa revalidation, or entering temporarily as crew members)” (https://en.wikipedia.org/wiki/Form_I-94)

An overview of this dataset is also outlined [here] (https://travel.trade.gov/research/programs/i94/description.asp)

Data files and formats:
- Data files are stored in SAS (proprietary?) sas7bdat format
- Per year a folder exists
- Per month a file exists (~500 GB)

Description file:
- A description file for the fields was included, named *I94_SAS_Labels_Descriptions.SAS*
- The file contains field descriptions for each column
- And it contains value constraints for some columns, namely: *i94cnty, i94port, i94mode, i94addr*

### B - I94 Immigration Data Data Gathering and first read

As Pandas has a method to import SAS data we will be using this mechanism. The following code will read a defined number of lines only due to performance reasons.

In [4]:
# Read in the data using read_sas() method
sas_file =  '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
sas_df = pd.DataFrame()
lines_imported = 0
max_lines=6000     # Set the desired line number here
for_lines=2000     # Set the desired lines for each cycle here

print('START reading SAS file ', sas_file)
# The method _read_sas()_ will read the files in chunks
for chunk in pd.read_sas(sas_file, 'sas7bdat', encoding="ISO-8859-1", chunksize=for_lines):
    last_lines = lines_imported + 1
    lines_imported = lines_imported + len(chunk)
    sas_df = sas_df.append(chunk)
    print('\t\t\tImporting lines from {} to {} of total {} lines'.format(last_lines, lines_imported, max_lines))
    if lines_imported >= max_lines:
        print('STOP reading SAS files')
        break

sas_df.head()

START reading SAS file  ../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat
			Importing lines from 1 to 2000 of total 6000 lines
			Importing lines from 2001 to 4000 of total 6000 lines
			Importing lines from 4001 to 6000 of total 6000 lines
STOP reading SAS files


,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,37.0,2.0,1.0,NaN,NaN,NaN,T,NaN,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,25.0,3.0,1.0,20130811,SEO,NaN,G,NaN,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,55.0,2.0,1.0,20160401,NaN,NaN,T,O,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,28.0,2.0,1.0,20160401,NaN,NaN,O,O,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,4.0,2.0,1.0,20160401,NaN,NaN,O,O,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


**Notes and Findings on First Read:**
* (N) We are not importing everything here, since the files amount to about 6GB in total
* (N) Using "chunksize" parameter and then breaking from the loop, so that we have handy **2.000 lines
* (F) In total 28 columns exist, 15 columns contain strings (object type) and 13 contain numbers (float64 type)
* At first sight one can already spot unfamiliar date columns (arrdate, depdate, etc.) with various datatypes
* Several rows have missing values
* Some columns contain obviously integer values but float64 was assigned
* Some categorical columns seem to exist

### C - Documentation Analysis

The workspace contains a field description file for the dataset named `I94_SAS_Labels_Descriptions.SAS`

The file seems pretty well structured, so I wrote a quick parser to automatically check the description file (see [SAS-Description-Parser](https://r766466c839826xjupyterlnnfq3jud.udacity-student-workspaces.com/lab/tree/SAS-Description-Parser.ipynb) for further details).

**Definitions**

| **Variable name** | **Data Type** | **Description** |
|---------------|---------------|---------------|
| i94yr | float64 | 4 digit year |
| i94mon | float64 | Numeric month |
| i94cit | float64 | This format shows all the valid and invalid codes for processing |
| i94res | float64 | This format shows all the valid and invalid codes for processing |
| i94port | object | This format shows all the valid and invalid codes for processing |
| arrdate | float64 | is the Arrival Date in the USA. It is a SAS date numeric field that apermament format has not been applied.  Please apply whichever date formatpermament format has not been applied.  Please apply whichever date format |
| i94mode | float64 | There are missing values as well as not reported (9) |
| i94addr | object | There is lots of invalid codes in this variable and the list belowThere is lots of invalid codes in this variable and the list below |
| depdate | float64 | is the Departure Date from the USA. It is a SAS date numeric field thata permament format has not been applied.  Please apply whichever date formata permament format has not been applied.  Please apply whichever date format |
| i94bir | float64 | Age of Respondent in Years |
| i94visa | float64 | Visa codes collapsed into three categories:1 = Business2 = Pleasure3 = Student*/ |
| count | float64 | Used for summary statistics |
| dtadfile | object | Character Date Field |
| visapost | object | Department of State where where Visa was issued |
| occup | object | Occupation that will be performed in U.S. |
| entdepa | object | Arrival Flag |
| entdepd | object | Departure Flag |
| entdepu | object | Update Flag |
| matflag | object | Match flag |
| biryear | float64 | 4 digit year of birth |
| dtaddto | object | Character Date Field |
| gender | object | Non |
| insnum | object | INS number |
| airline | object | Airline used to arrive in U.S. |
| admnum | float64 | Admission Number |
| fltno | object | Flight number of Airline used to arrive in U.S. |
| visatype | object | Class of admission legally admitting the non |

**Findings on value constraints**

Columns `i94cnty, i94port, i94mode, i94addr` have value constraints (lists with allowed entry values) which are outlined here:
* `i94cnty` contains country short codes and their corresponding state names
* `i94port`contains port/airport codes from various cities
    * There doesn't seem to be a specific selection criteria
    * Although most of the codes are cities in the US we also see city codes from Europe and Asia
* `i94mode` is a code for the way of travelling (by Air, by Sea or by Land) or unknown
* `i94addr`is a code for the state in which this immigrants temporary address is located (aka "First Intended Address")


### D - Analysis of numeric columns
The Pandas describe() function creates a basic set of descriptive statistics for each numeric column in the data frame.

In [4]:
quantify_data(sas_df, ['numbers'])

Running Data Quantifier with parameter:  numbers

Quantifying NUMERIC data types in columns:
 cicid, i94yr, i94mon, i94cit, i94res, arrdate, i94mode, depdate, i94bir, i94visa, count, biryear, admnum


,cicid,i94yr,i94mon,i94cit,i94res,arrdate,i94mode,depdate,i94bir,i94visa,count,biryear,admnum
count,2000.000000,2000.0,2000.0,2000.00000,2000.000000,2000.000000,1999.000000,1927.000000,2000.000000,2000.0000,2000.0,2000.000000,2.000000e+03
mean,1146.768000,2016.0,4.0,104.16250,111.666000,20545.017000,1.006003,20556.293202,38.968000,1.9015,1.0,1977.032000,5.909467e+10
std,650.874837,0.0,0.0,13.60072,54.716217,0.640247,0.077265,16.081589,17.266993,0.3144,0.0,17.266993,1.507924e+10
min,6.000000,2016.0,4.0,101.00000,101.000000,20545.000000,1.000000,20546.000000,0.000000,1.0000,1.0,1929.000000,6.644910e+08
25%,565.750000,2016.0,4.0,103.00000,103.000000,20545.000000,1.000000,20550.000000,26.000000,2.0000,1.0,1964.000000,5.542381e+10
50%,1159.500000,2016.0,4.0,104.00000,104.000000,20545.000000,1.000000,20552.000000,39.000000,2.0000,1.0,1977.000000,5.543718e+10
75%,1678.250000,2016.0,4.0,104.00000,104.000000,20545.000000,1.000000,20558.000000,52.000000,2.0000,1.0,1990.000000,5.545536e+10
max,2302.000000,2016.0,4.0,692.00000,692.000000,20573.000000,2.000000,20704.000000,87.000000,3.0000,1.0,2016.000000,9.251651e+10
Unique,2000.000000,1.0,1.0,8.00000,36.000000,3.000000,2.000000,80.000000,83.000000,3.0000,1.0,83.000000,2.000000e+03




Data Quantification Done




**Summary on numeric data:**

| **Column Name** | **Commends and findings:**                  | **Recommended action items** |
|------------|--------------------------------------------------|----------------------------|
| `cicid` |is unique for all 2.000 lines (check `len(sas_df['cicid'].unique())`) and appears to be the primary key for each record | |
| | The following columns appear to indicate datetime related values: |
| `i94yr` |indicating the year the I94 form was filled and 'i94mon' indicating the month | |
| `arrdate` |is the immigrants arrival date | |
| `depdate` |the date of the immigrants (planned) departure | |
| `dtadfile` |is the date on which the form was entered into the database | |
| `dtaddto` |is the date the immigrant is admissioned to stay in the US | |
| `i94mode` | has already been identified as a category variable, the integers here are just codes indicating if the immigrant travelled by Land, Air or Sea (or unknown) | |
| `i94visa` | was not identified correctly by my parser it seems, it has value constraints (* 1 = Business, 2 = Pleasure,3 = Student)  | |
| `i94cit` and `i94res` | are again not numeric but indicate the immigrant's countries of citizenship ("cit") and residence (res) | |
|`admnum` | is the admission number | |
|`i94bir` |appears to be the immigrant's age at the time of admission (in other words it's the time delta between `i94yr`and `biryear` | |
| `biryear` |marks the immigrants birthyear | |
| `count` |is for statistical purposes according to the description | |

### E - Quantitative Analysis of non-numeric data
Measuring the number of NaN entries and unique values

In [6]:
quantify_data(sas_df, ['object'])

Running Data Quantifier with parameter:  object  and example threshhold is  10

Quantifying NON-NUMERIC data types in columns:
 i94port, i94addr, dtadfile, visapost, occup, entdepa, entdepd, entdepu, matflag, dtaddto, gender, insnum, airline, fltno, visatype


,i94port,i94addr,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,dtaddto,gender,insnum,airline,fltno,visatype
Datatype,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
Lines,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000
Non-Null,6000,5846,5999,675,3,6000,5821,2,5821,5999,5247,0,5998,5999,6000
NaN,0,154,1,5325,5997,0,179,5998,179,1,753,6000,2,1,0
Fill-%,100,97.4333,99.9833,11.25,0.05,100,97.0167,0.0333333,97.0167,99.9833,87.45,0,99.9667,99.9833,100
Unique,51,56,2,45,3,6,8,2,1,33,2,0,83,455,8
Uniq-%,0.85,0.933333,0.0333333,0.75,0.05,0.1,0.133333,0.0333333,0.0166667,0.55,0.0333333,0,1.38333,7.58333,0.133333


Columns with missing values:  i94addr,dtadfile,visapost,occup,entdepd,entdepu,matflag,dtaddto,gender,insnum,airline,fltno 

Unique values in column 'dtadfile': 
 [nan '20130811' '20160401'] 

Unique values in column 'occup': 
 [nan 'ELT' 'PHS' 'EXA'] 

Unique values in column 'entdepa': 
 ['T' 'G' 'O' 'H' 'U' 'B'] 

Unique values in column 'entdepd': 
 [nan 'O' 'K' 'I' 'Q' 'R' 'N' 'M' 'J'] 

Unique values in column 'entdepu': 
 ['U' 'Y' nan] 

Unique values in column 'matflag': 
 [nan 'M'] 

Unique values in column 'gender': 
 [nan 'M' 'F'] 

Unique values in column 'insnum': 
 [nan] 

Unique values in column 'visatype': 
 ['B2' 'F1' 'B1' 'WT' 'WB' 'E2' 'I' 'F2'] 



Data Quantification Done




**Summary on non-numeric data:**

| **Column Name** | **Commends and findings:**                  | **Recommended action items** |
|------------|--------------------------------------------------|----------------------------|
| several | columns have missing values (i94addr,dtadfile,visapost,occup,entdepd,entdepu,matflag,dtaddto,gender,insnum,airline,fltno) | |


In [9]:
new_df = sas_df.copy()

to_datetime_cols = ['arrdate', 'depdate']

to_int_cols = ['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res',\
               'i94port', 'i94mode', 'i94bir', 'i94visa', 'count',\
               'biryear'
              ]

to_string_cols = []

sas_df_category_columns = ['i94mode', 'gender', 'occup']

sas_df_type = {
'cicid':'int',
'i94yr':'int',
'i94mon':'int',
'i94cit':'int',
'i94res':'int',
'i94port':'object',
'arrdate':'int',
'i94mode':'object',
'i94addr':'object',
'depdate':'int',
'i94bir':'float64',
'i94visa':'float64',
'count':'int',
'dtadfile':'object',
'visapost':'object',
'occup':'object',
'entdepa':'object',
'entdepd':'object',
'entdepu':'object',
'matflag':'object',
'biryear':'int',
'dtaddto':'object',
'gender':'object',
'insnum':'object',
'airline':'object',
'admnum':'float64',
'fltno':'object',
'visatype':'category'
}

# Import SAS definition file
from sas_helpers import get_sas_definitions
sasdefs = get_sas_definitions('I94_SAS_Labels_Descriptions.SAS')

# Note: IN SAS dates are a special case of numeric values. Each day is assigned a specific numeric value starting from 1st January 1960.
# Source: https://www.tutorialspoint.com/sas/sas_dates_times.htm

# Convert to Integer
for col in to_int_cols:
    pass

# Transform date time columns:
for col in sas_df.select_dtypes(include=['float64']):
    # From SAS date to Python datetime64
    if col in ['arrdate', 'depdate']:
        new_df[[key]] = pd.to_datetime(new_df[key], infer_datetime_format=True, errors='coerce', origin=pd.Timestamp('1960-01-01'))
        #sas_df[[key]] = sas_df[key].values.strftime('%Y-%m-%d')
        #new_df[[key]] = pd.to_datetime(sas_df[key], unit='D', origin=pd.Timestamp('1960-01-01'), errors='coerce')
    # From Float64 to Integer
    if col in ['i94yr', 'i94mon']:
        new_df[[key]] = new_df[[key]].astype(int)
    # From String "YYYYMMDD" to datetime
    if col in ['dtadfile']:
        new_df[[key]] = pd.to_numeric(new_df[key], errors='coerce')
        new_df[[key]] = new_df[[key]].fillna(0)
    if col in ['dtaddto']:
        new_df[[key]] = pd.to_numeric(new_df[key], errors='coerce')
        new_df[[key]] = new_df[[key]].fillna(0)


new_df.head()

AttributeError: 'dict' object has no attribute 'dtypes'

In [10]:
sas_df['admnum'].describe()

count    2.000000e+03
mean     5.909467e+10
std      1.507924e+10
min      6.644910e+08
25%      5.542381e+10
50%      5.543718e+10
75%      5.545536e+10
max      9.251651e+10
Name: admnum, dtype: float64

In [3]:
	
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [11]:
#write to parquet
df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.